In [6]:
import pandas as pd

In [7]:
#class below is for holding single id and single url
class holder:
    def __init__(self,ID = 0,url = None):
        self.ID = ID
        self.url = url
    def __eq__(self,obj):
        return str(self.ID)+self.url == str(obj.ID)+obj.url
    def __str__(self):
        return str(self.ID)+' '+self.url


In [8]:
#class below is for holding term frequency and list of holder classess
class termValue:
    def __init__(self):
        self.frequency = 0
        self.holderList = []
    def addHolder(self,newHolder):
        if newHolder not in self.holderList:
            self.holderList.append(newHolder)

In [9]:
#function below is for calculating frequency of each term
def frequencyCalc(myDictionary):
    for i in myDictionary:
        myDictionary[i].frequency = len(myDictionary[i].holderList)

In [10]:
#algorithm below is for retrieval of the dictionary
myFile = open('dictionary.txt',encoding = 'UTF-8')
theDictionary = {}
for i in myFile:
    res = i.split('$')
    if res[0] not in theDictionary:
        theDictionary[res[0]] = termValue()
        myHolder = holder(res[1],res[2])
        theDictionary[res[0]].addHolder(myHolder)
    else:
        myHolder = holder(res[1],res[2])
        theDictionary[res[0]].addHolder(myHolder)
myFile.close()

In [11]:
frequencyCalc(theDictionary)

In [12]:
#list below is just for saving list of words
wordsList = []
for i in theDictionary:
    wordsList.append(i)